# 2.0 Coordinates to Postcode
Convert the longitude and latitude in the Zoopla data to postcodes. Unfortunately we don't have the postcode or house number in the Zoopla data. So we will infer this from the property longitude and latitude. Datasets of all postcodes for a given area, including their mean latitude and longitude are available from Source: https://www.doogal.co.uk/AdministrativeAreas, which we refer to as the mapping file. We will then use the inferred property postcode to join other data (e.g. deprivation data, flood risk data, EPC data (averages for that postcode and road) and government historic property prices (averages for that postcode, house type, year, etc).

In [1]:
from collections import Counter
import os
import numpy as np
import pandas as pd
import re
pd.set_option('display.max_columns', 100)

### Read in Zoopla data and mapping files that convert co-ordinates to postcodes
Also concatenate the Nuneaton and Hinckley files

In [2]:
DATA_FOLDER = os.path.join('data', 'raw')
SAVE_FOLDER = os.path.join('data', 'processed')

In [3]:
zoopla_df_filenames = [fname for fname in os.listdir(DATA_FOLDER) if fname.startswith('zoopla_properties')]

zoopla_df_list = []

for filename in zoopla_df_filenames:
    zoopla_df_list.append(pd.read_csv(os.path.join(DATA_FOLDER, filename)))
    
zoopla_df = pd.concat(zoopla_df_list).drop_duplicates(subset='listing_id')

In [4]:
print(zoopla_df.shape)
display(zoopla_df.head())

(1466, 28)


,category,agent_logo,country_code,property_type,num_bedrooms,num_bathrooms,agent_phone,county,street_name,post_town,listing_status,outcode,agent_address,details_url,num_recepts,listing_id,displayable_address,image_url,first_published_date,price_modifier,agent_name,longitude,price,description,floor_plan,country,latitude,last_published_date
0,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Terraced house,2,2,024 7662 0227,Warwickshire,Tomkinson Road,Nuneaton,sale,CV10,"Bond Gate Chambers, Bond Gate, Nuneaton",https://www.zoopla.co.uk/for-sale/details/6446...,2,64462156,"Tomkinson Road, Nuneaton CV10",https://lid.zoocdn.com/354/255/260b0fbca9f9ea4...,2023-04-22 08:33:46,NaN,Pointons Estate Agents,-1.487903,165000.0,***buy to let investors only tenants in situ ...,NaN,England,52.521416,2023-04-22 08:50:53
1,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Detached house,4,3,024 7513 1114,Warwickshire,277 The Long Shoot,Nuneaton,sale,CV11,"1a - 4a Bond Gate Chambers, Nuneaton",https://www.zoopla.co.uk/for-sale/details/6445...,4,64459560,"The Long Shoot, Nuneaton CV11",https://lid.zoocdn.com/354/255/abfc27ca46df8d6...,2023-04-21 17:31:08,NaN,Tom Bates Estate Agents,-1.421078,525000.0,""" incedible detached home with cinema room "" W...",NaN,England,52.535797,2023-04-21 20:28:04
2,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Detached bungalow,3,1,024 7662 0227,Warwickshire,Crossing Gates,Nuneaton,sale,CV11,"Bond Gate Chambers, Bond Gate, Nuneaton",https://www.zoopla.co.uk/for-sale/details/6445...,1,64458751,"Crossing Gates, Nuneaton CV11",https://lid.zoocdn.com/354/255/c9d88b69e108d57...,2023-04-21 16:27:48,NaN,Pointons Estate Agents,-1.454462,340000.0,Pointons are delighted to offer for sale show...,NaN,England,52.525795,2023-04-21 16:39:38
3,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Detached house,4,2,024 7511 9743,Warwickshire,Cliveden Walk,Nuneaton,sale,CV11,"4 Hearsall Lane, Earlsdon, Coventry",https://www.zoopla.co.uk/for-sale/details/6445...,1,64456960,"Cliveden Walk, Maple Park, Nuneaton CV11",https://lid.zoocdn.com/354/255/2f9a207a5b6ece8...,2023-04-21 14:35:46,NaN,Maison Estates Ltd,-1.450420,420000.0,***grande & imposing 4 bedroom detached house...,NaN,England,52.502000,2023-04-21 14:35:46
4,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Detached house,3,0,0333 016 5458,Warwickshire,Moor Road,Nuneaton,sale,CV10,"St George's House, 56 Peter Street, Manchester",https://www.zoopla.co.uk/for-sale/details/6445...,0,64451326,"Moor Road, Nuneaton CV10",https://lid.zoocdn.com/354/255/e44f87a4df01d35...,2023-04-21 04:24:39,NaN,Express Estate Agency,-1.524742,300000.0,We are proud to present this Three Bedroom Det...,NaN,England,52.540840,2023-04-21 04:24:39


In [5]:
mapping_df_filename_nuneaton = 'Nuneaton and Bedworth postcodes.csv'
mapping_df_filename_hinckley = 'Hinckley and Bosworth postcodes.csv'

mapping_df_nuneaton = pd.read_csv(os.path.join(DATA_FOLDER, mapping_df_filename_nuneaton))
mapping_df_hinckley = pd.read_csv(os.path.join(DATA_FOLDER, mapping_df_filename_hinckley))
mapping_df = pd.concat([mapping_df_nuneaton, mapping_df_hinckley]).drop_duplicates()

In [6]:
print(mapping_df.shape)
display(mapping_df.head())

(6886, 17)


,Postcode,In Use?,Latitude,Longitude,Easting,Northing,Grid Ref,Ward,Parish,Introduced,Terminated,Altitude,Country,Last Updated,Quality,LSOA Code,LSOA Name
0,CV10 0AA,Yes,52.526750,-1.460760,436681,292234,SP366922,St. Nicolas,"Nuneaton and Bedworth, unparished area",1980-01-01,NaN,85,England,2022-11-25,Within the building of the matched address clo...,E01031102,Nuneaton and Bedworth 003C
1,CV10 0AB,Yes,52.527391,-1.459293,436780,292306,SP367923,St. Nicolas,"Nuneaton and Bedworth, unparished area",1980-01-01,NaN,86,England,2022-11-25,Within the building of the matched address clo...,E01031102,Nuneaton and Bedworth 003C
2,CV10 0AD,Yes,52.527600,-1.461965,436599,292328,SP365923,St. Nicolas,"Nuneaton and Bedworth, unparished area",1980-01-01,NaN,86,England,2022-11-25,Within the building of the matched address clo...,E01031102,Nuneaton and Bedworth 003C
3,CV10 0AE,No,52.526840,-1.462793,436543,292243,SP365922,St. Nicolas,"Nuneaton and Bedworth, unparished area",1980-01-01,2009-11-01,86,England,2022-11-25,Within the building of the matched address clo...,E01031102,Nuneaton and Bedworth 003C
4,CV10 0AF,Yes,52.538271,-1.467776,436196,293512,SP361935,Weddington,"Nuneaton and Bedworth, unparished area",2006-04-01,NaN,84,England,2022-11-25,Within the building of the matched address clo...,E01031113,Nuneaton and Bedworth 001D


### Map latitude and longitude to postcode

Create function that takes the longitude and latitude from the Zoopla data and find the closest one in the mapping data, returning the corresponding postcode in the mapping file.

In [7]:
def get_closest_postcode(latitude, longitude, map_df):
    
    """
    Find the nearest longitude and latitude in mapping file and get the postcode.
    This uses the Euclidean (rather than Haversine) distance, since the distances will be small
    such that the Earth's curvature need not be considered.
    An alternative could be to use geopy.reverse() to convert coordinates to postcodes.
    
    Parameters
    ----------
    - latitude (float): latitude of the Zoopla property
    - longitude (float): longitude of the Zoopla property
    - map_df (pandas.DataFrame): dataframe that maps coordinates to postcodes
    
    Returns
    -------
    - postcode (string): postcode closely mapping the Zoopla property
    """
    
    # squared euclidean distance between Zoopla property and each mapping dataset postcode mean
    # this uses array broadcasting in numpy
    sq_distances = (np.array(latitude) - np.array(map_df['Latitude']))**2 + \
        (np.array(longitude) - np.array(map_df['Longitude']))**2 
    
    min_sq_distance_row = np.argmin(sq_distances)
    
    return map_df.iloc[min_sq_distance_row]['Postcode']
    

In [8]:
zoopla_df['postcode_test_1'] = zoopla_df[['latitude', 'longitude']].apply(lambda x: get_closest_postcode(x[0], x[1], mapping_df), axis=1)

In [9]:
zoopla_df[['latitude', 'longitude', 'postcode_test_1']].head(10)

,latitude,longitude,postcode_test_1
0,52.521416,-1.487903,CV10 8BN
1,52.535797,-1.421078,CV11 6JN
2,52.525795,-1.454462,CV11 6GR
3,52.502000,-1.450420,CV11 4GD
4,52.540840,-1.524742,CV10 0PL
5,52.526566,-1.515683,CV10 9JX
6,52.529346,-1.460990,CV10 0BA
7,52.507340,-1.480143,CV10 7PN
8,52.572170,-1.412421,CV13 6HJ
9,52.533386,-1.461711,CV10 0BW


This works OK but sometimes we get an adjacent postcode to the one we want. Instead, let's try combining it with the road name by loading the EPC data which has postcodes and names, and joining to the Zoopla data on the street name.

### Try getting postcode from road name using EPC data
Data available from https://epc.opendatacommunities.org/domestic/search  
Concatenate the Nuneaton and Hinckley EPC data

In [10]:
def get_most_common_postcode(postcode_list):
    
    """
    Return the most frequent value in a list
    """
    
    if not postcode_list:
        return None
    else:
        postcode_counter = Counter(postcode_list)
        return postcode_counter.most_common()[0][0]
    

In [11]:
def get_street_name(address_1, address_2):
    
    """
    Get street name from first two street address fields
    """
    
    street_and_road = re.compile(r'^\d,\s+')
    
    # if street name starts with a number (maybe followed by comma) and a space, likely next part is street name
    if street_and_road.match(address_1):
        street = re.split(street_and_road, address_1)[1].lower()
        
    # otherwise choose the second part of the address as the street name
    else:
        street = str(address_2).lower()
        
    return street


In [12]:
def get_postcode(latitude, longitude, street_name, method='closest'):
    
    """
    Convert longitude and latitude into a post code, using the street name to narrow 
    the possible post codes down. Two methods possible, discussed below.
    
    Parameters
    ----------
    - latitude (float): latitude of the Zoopla property
    - longitude (float): longitude of the Zoopla property
    - street_name (string): the street name for the Zoopla property
    - method (string) default='closest': algorithm for choosing the postcode
       - closest: uses Euclidean distance to calculate the
         nearest longitude and latitude in the mapping file to the Zoopla property and gets the postcode
       - frequency: gets the most common postcode for the property's street name
    
    Returns
    -------
    - postcode (string): postcode closely mapping the Zoopla property
    """
    
    # get all possible postcodes for the street name
    possible_postcodes = list(epc_df[epc_df['Street'].str.lower()==str(street_name).lower()]['POSTCODE'])
    
    if method == 'frequency':
        
        final_postcode = get_most_common_postcode(possible_postcodes)
        
        # if there is no most common postcode in the EPC data, use the 'closest' algorithm instead
        if not final_postcode:
            method = 'closest'
        
    if method == 'closest':
        
        possible_postcodes = set(possible_postcodes)
        
        # if set is empty, try all possible postcodes and find nearest one
        if not possible_postcodes:
            final_postcode = get_closest_postcode(latitude, longitude, mapping_df)
        
        # if set is filled, limit search to the postcodes in the set
        else:

            # get mapping dataframe of just the postcodes in the above set
            mapping_df_temp = mapping_df[mapping_df['Postcode'].isin(possible_postcodes)]
            
            # where mapping dataframe filled, otherwise, choose the postcode from the set 
            # whose location is closest to the property longitude and latitude
            if not mapping_df_temp.empty:
                final_postcode = get_closest_postcode(latitude, longitude, mapping_df_temp)
            
            # if the postcodes are not found in the mapping file (likely due to
            # that postcode falling outside the geo boundary of the mapping file),
            # set the postcode to the first element in the possible_postcodes set 
            else:
                final_postcode = sorted(possible_postcodes)[0]
            

    return final_postcode
  

In [13]:
epc_filename_nuneaton = 'epcs_nuneaton.csv'
epc_filename_hinckley = 'epcs_hinckley.csv'

epc_df_nuneaton = pd.read_csv(os.path.join(DATA_FOLDER, epc_filename_nuneaton), dtype=str)
epc_df_hinckley = pd.read_csv(os.path.join(DATA_FOLDER, epc_filename_hinckley), dtype=str)
epc_df = pd.concat([epc_df_nuneaton, epc_df_hinckley]).drop_duplicates()

In [14]:
print(epc_df.shape)
display(epc_df.head())

(91212, 92)


,LMK_KEY,ADDRESS1,ADDRESS2,ADDRESS3,POSTCODE,BUILDING_REFERENCE_NUMBER,CURRENT_ENERGY_RATING,POTENTIAL_ENERGY_RATING,CURRENT_ENERGY_EFFICIENCY,POTENTIAL_ENERGY_EFFICIENCY,PROPERTY_TYPE,BUILT_FORM,INSPECTION_DATE,LOCAL_AUTHORITY,CONSTITUENCY,COUNTY,LODGEMENT_DATE,TRANSACTION_TYPE,ENVIRONMENT_IMPACT_CURRENT,ENVIRONMENT_IMPACT_POTENTIAL,ENERGY_CONSUMPTION_CURRENT,ENERGY_CONSUMPTION_POTENTIAL,CO2_EMISSIONS_CURRENT,CO2_EMISS_CURR_PER_FLOOR_AREA,CO2_EMISSIONS_POTENTIAL,LIGHTING_COST_CURRENT,LIGHTING_COST_POTENTIAL,HEATING_COST_CURRENT,HEATING_COST_POTENTIAL,HOT_WATER_COST_CURRENT,HOT_WATER_COST_POTENTIAL,TOTAL_FLOOR_AREA,ENERGY_TARIFF,MAINS_GAS_FLAG,FLOOR_LEVEL,FLAT_TOP_STOREY,FLAT_STOREY_COUNT,MAIN_HEATING_CONTROLS,MULTI_GLAZE_PROPORTION,GLAZED_TYPE,GLAZED_AREA,EXTENSION_COUNT,NUMBER_HABITABLE_ROOMS,NUMBER_HEATED_ROOMS,LOW_ENERGY_LIGHTING,NUMBER_OPEN_FIREPLACES,HOTWATER_DESCRIPTION,HOT_WATER_ENERGY_EFF,HOT_WATER_ENV_EFF,FLOOR_DESCRIPTION,FLOOR_ENERGY_EFF,FLOOR_ENV_EFF,WINDOWS_DESCRIPTION,WINDOWS_ENERGY_EFF,WINDOWS_ENV_EFF,WALLS_DESCRIPTION,WALLS_ENERGY_EFF,WALLS_ENV_EFF,SECONDHEAT_DESCRIPTION,SHEATING_ENERGY_EFF,SHEATING_ENV_EFF,ROOF_DESCRIPTION,ROOF_ENERGY_EFF,ROOF_ENV_EFF,MAINHEAT_DESCRIPTION,MAINHEAT_ENERGY_EFF,MAINHEAT_ENV_EFF,MAINHEATCONT_DESCRIPTION,MAINHEATC_ENERGY_EFF,MAINHEATC_ENV_EFF,LIGHTING_DESCRIPTION,LIGHTING_ENERGY_EFF,LIGHTING_ENV_EFF,MAIN_FUEL,WIND_TURBINE_COUNT,HEAT_LOSS_CORRIDOR,UNHEATED_CORRIDOR_LENGTH,FLOOR_HEIGHT,PHOTO_SUPPLY,SOLAR_WATER_HEATING_FLAG,MECHANICAL_VENTILATION,ADDRESS,LOCAL_AUTHORITY_LABEL,CONSTITUENCY_LABEL,POSTTOWN,CONSTRUCTION_AGE_BAND,LODGEMENT_DATETIME,TENURE,FIXED_LIGHTING_OUTLETS_COUNT,LOW_ENERGY_FIXED_LIGHT_COUNT,UPRN,UPRN_SOURCE
0,1230178709552014110416461495049124,"71, Riversley Road",NaN,NaN,CV11 5QT,1404959278,E,D,45,64,Maisonette,End-Terrace,2014-11-01,E07000219,E14000868,Warwickshire,2014-11-04,none of the above,42,62,384,236,5.1,74,3.1,96,51,989,616,110,110,69.0,dual,Y,1st,Y,NaN,2102,100,double glazing installed before 2002,Normal,1,4,4,11,0,From main system,Good,Good,(other premises below),NaN,NaN,Fully double glazed,Average,Average,"Solid brick, as built, no insulation (assumed)",Very Poor,Very Poor,None,NaN,NaN,"Pitched, no insulation (assumed)",Very Poor,Very Poor,"Boiler and radiators, mains gas",Good,Good,"Programmer, no room thermostat",Very Poor,Very Poor,Low energy lighting in 11% of fixed outlets,Poor,Poor,mains gas (not community),0,no corridor,NaN,NaN,0.0,NaN,natural,"71, Riversley Road",Nuneaton and Bedworth,Nuneaton,NUNEATON,England and Wales: 1900-1929,2014-11-04 16:46:14,owner-occupied,9,1,100070163319,Address Matched
1,868018689262012122112363153798182,"23, Arden Road",Bulkington,NaN,CV12 9JJ,8533893078,D,B,59,85,House,Detached,2012-12-21,E07000219,E14000905,Warwickshire,2012-12-21,FiT application,55,85,251,80,4.6,48,1.5,64,64,795,441,86,63,96.0,Single,Y,NODATA!,NaN,NaN,2106,100,double glazing installed during or after 2002,Normal,1,7,7,75,0,From main system,Good,Good,"Solid, no insulation (assumed)",NaN,NaN,Fully double glazed,Good,Good,"Solid brick, as built, no insulation (assumed)",Very Poor,Very Poor,"Room heaters, mains gas",NaN,NaN,"Pitched, 150 mm loft insulation",Good,Good,"Boiler and radiators, mains gas",Good,Good,"Programmer, room thermostat and TRVs",Good,Good,Low energy lighting in 75% of fixed outlets,Very Good,Very Good,mains gas (not community),0,NO DATA!,NaN,NaN,0.0,NaN,natural,"23, Arden Road, Bulkington",Nuneaton and Bedworth,Rugby,BEDWORTH,England and Wales: 1900-1929,2012-12-21 12:36:31,owner-occupied,12,9,100070135974,Address Matched
2,1008849941732013092016263284978300,"249, Lutterworth Road",NaN,NaN,CV11 6PU,4044393178,E,C,50,76,Bungalow,Detached,2013-09-16,E07000219,E14000868,Warwickshire,2013-09-20,marketed sale,48,75,273,126,7.1,52,3.3,112,73,1292,857,172,84,136.0,dual,Y,NODATA!,NaN,NaN,2106,95,double glazing installed before 2002,Normal,1,6,6,47,1,From main system,Average,Average,"Solid, no insulation (assumed)",NaN,NaN,Mostly double gla

In [15]:
# get street name from addresses
epc_df['Street'] = epc_df[['ADDRESS1', 'ADDRESS2']].apply(lambda x: get_street_name(x[0], x[1]), axis=1)

In [16]:
# try both 'closest' and 'frequency' method to get postcode
zoopla_df['postcode_test_2'] = zoopla_df[['latitude', 'longitude', 'street_name']].apply(lambda x: get_postcode(x[0], x[1], x[2], method='closest'), axis=1)
zoopla_df['postcode_test_3'] = zoopla_df[['latitude', 'longitude', 'street_name']].apply(lambda x: get_postcode(x[0], x[1], x[2], method='frequency'), axis=1)

In [17]:
zoopla_df[['latitude', 'longitude', 'postcode_test_1', 'postcode_test_2', 'postcode_test_3']].head(30)

,latitude,longitude,postcode_test_1,postcode_test_2,postcode_test_3
0,52.521416,-1.487903,CV10 8BN,CV10 8DU,CV10 8DU
1,52.535797,-1.421078,CV11 6JN,CV11 6JN,CV11 6JN
2,52.525795,-1.454462,CV11 6GR,CV11 6GR,CV11 6GR
3,52.502000,-1.450420,CV11 4GD,CV11 4XJ,CV11 4XJ
4,52.540840,-1.524742,CV10 0PL,CV10 0PL,CV10 0PL
5,52.526566,-1.515683,CV10 9JX,CV10 9JX,CV10 9JX
6,52.529346,-1.460990,CV10 0BA,CV10 0BB,CV10 0BB
7,52.507340,-1.480143,CV10 7PN,CV10 7PN,CV10 7PN
8,52.572170,-1.412421,CV13 6HJ,CV13 6HJ,CV13 6HJ
9,52.533386,-1.461711,CV10 0BW,CV10 0BW,CV10 0BH


From checks on Google, the 'closest' method is most accurate so will be used for this dataset

In [18]:
zoopla_df.drop(columns=['postcode_test_1', 'postcode_test_3'], inplace=True)
zoopla_df = zoopla_df.rename(columns={'postcode_test_2': 'postcode'})

### Get the parish from the postcode

In [19]:
def get_parish(postcode, town):
    
    """
    Get the parish of the Zoopla property from the mapping dataframe (based on postcode and town)
    """
    
    try:
        return mapping_df[mapping_df['Postcode']==postcode].iloc[0]['Parish']
    except IndexError:
        if town in ['Nuneaton', 'Bedworth', 'Atherstone']:
            return "Nuneaton and Bedworth, unparished area"
        elif town in ['Hinckley', 'Bosworth']:
            return "Hinckley and Bosworth, unparished area"
        else:
            return "Other/Unknown"

In [20]:
zoopla_df['post_town'].value_counts()

Nuneaton      968
Hinckley      478
Atherstone     11
Bedworth        8
Derby           1
Name: post_town, dtype: int64

In [21]:
zoopla_df['parish'] = zoopla_df[['postcode', 'post_town']].apply(lambda x: get_parish(x[0], x[1]), axis=1)

In [22]:
zoopla_df['parish'].value_counts()

Nuneaton and Bedworth, unparished area    936
Hinckley and Bosworth, unparished area    459
Stoke Golding                              26
Burbage                                    21
Higham on the Hill                          9
Witherley                                   7
Earl Shilton                                2
Desford                                     1
Market Bosworth                             1
Barwell                                     1
Nailstone                                   1
Barlestone                                  1
Sheepy                                      1
Name: parish, dtype: int64

Maybe not such a useful feature but keep for now!

### Save to csv file

In [23]:
zoopla_df.head()

,category,agent_logo,country_code,property_type,num_bedrooms,num_bathrooms,agent_phone,county,street_name,post_town,listing_status,outcode,agent_address,details_url,num_recepts,listing_id,displayable_address,image_url,first_published_date,price_modifier,agent_name,longitude,price,description,floor_plan,country,latitude,last_published_date,postcode,parish
0,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Terraced house,2,2,024 7662 0227,Warwickshire,Tomkinson Road,Nuneaton,sale,CV10,"Bond Gate Chambers, Bond Gate, Nuneaton",https://www.zoopla.co.uk/for-sale/details/6446...,2,64462156,"Tomkinson Road, Nuneaton CV10",https://lid.zoocdn.com/354/255/260b0fbca9f9ea4...,2023-04-22 08:33:46,NaN,Pointons Estate Agents,-1.487903,165000.0,***buy to let investors only tenants in situ ...,NaN,England,52.521416,2023-04-22 08:50:53,CV10 8DU,"Nuneaton and Bedworth, unparished area"
1,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Detached house,4,3,024 7513 1114,Warwickshire,277 The Long Shoot,Nuneaton,sale,CV11,"1a - 4a Bond Gate Chambers, Nuneaton",https://www.zoopla.co.uk/for-sale/details/6445...,4,64459560,"The Long Shoot, Nuneaton CV11",https://lid.zoocdn.com/354/255/abfc27ca46df8d6...,2023-04-21 17:31:08,NaN,Tom Bates Estate Agents,-1.421078,525000.0,""" incedible detached home with cinema room "" W...",NaN,England,52.535797,2023-04-21 20:28:04,CV11 6JN,"Nuneaton and Bedworth, unparished area"
2,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Detached bungalow,3,1,024 7662 0227,Warwickshire,Crossing Gates,Nuneaton,sale,CV11,"Bond Gate Chambers, Bond Gate, Nuneaton",https://www.zoopla.co.uk/for-sale/details/6445...,1,64458751,"Crossing Gates, Nuneaton CV11",https://lid.zoocdn.com/354/255/c9d88b69e108d57...,2023-04-21 16:27:48,NaN,Pointons Estate Agents,-1.454462,340000.0,Pointons are delighted to offer for sale show...,NaN,England,52.525795,2023-04-21 16:39:38,CV11 6GR,"Nuneaton and Bedworth, unparished area"
3,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Detached house,4,2,024 7511 9743,Warwickshire,Cliveden Walk,Nuneaton,sale,CV11,"4 Hearsall Lane, Earlsdon, Coventry",https://www.zoopla.co.uk/for-sale/details/6445...,1,64456960,"Cliveden Walk, Maple Park, Nuneaton CV11",https://lid.zoocdn.com/354/255/2f9a207a5b6ece8...,2023-04-21 14:35:46,NaN,Maison Estates Ltd,-1.450420,420000.0,***grande & imposing 4 bedroom detached house...,NaN,England,52.502000,2023-04-21 14:35:46,CV11 4XJ,"Nuneaton and Bedworth, unparished area"
4,Residential,https://st.zoocdn.com/zoopla_static_agent_logo...,gb,Detached house,3,0,0333 016 5458,Warwickshire,Moor Road,Nuneaton,sale,CV10,"St George's House, 56 Peter Street, Manchester",https://www.zoopla.co.uk/for-sale/details/6445...,0,64451326,"Moor Road, Nuneaton CV10",https://lid.zoocdn.com/354/255/e44f87a4df01d35...,2023-04-21 04:24:39,NaN,Express Estate Agency,-1.524742,300000.0,We are proud to present this Three Bedroom Det...,NaN,England,52.540840,2023-04-21 04:24:39,CV10 0PL,"Nuneaton and Bedworth, unparished area"


In [24]:
try:
    os.mkdir(SAVE_FOLDER)
except OSError:
    pass

save_file = os.path.join(SAVE_FOLDER, 'zoopla_properties_with_postcode.csv')
    
zoopla_df.to_csv(save_file, index=False)